## Traditional machine learning
Feature(s): 'SMILES' column

- 'SMILES' column

Target: deltaG4_bin

- Binary class

In [6]:
import sys
import os
from sklearn.model_selection import cross_val_score
from sklearn.base import clone
from loguru import logger

logger.remove()
logger.add(sys.stderr, level="WARNING")
# Append the parent directory of your package to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..')))

In [7]:
import pandas as pd
import numpy as np
import zipfile

path_to_dataset = 'NiCatalysis.csv'
csv_filename = 'NiCatalysis.csv'

# Open the file, Correct the encoding and sep if necessary
if path_to_dataset.endswith('.zip'):
    with zipfile.ZipFile(path_to_dataset, 'r') as z:
        # Open the CSV file within the ZIP file
        with z.open(csv_filename) as f:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(f, sep=',', on_bad_lines='warn', index_col = 0)
else:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(path_to_dataset, sep=',', on_bad_lines='warn', index_col = 0)

df = df.rename(columns={'Smiles': 'SMILES'})

print('Count of unique smiles:', df.SMILES.unique().shape[0])
print('Count of all of the smiles:', df.shape[0])


Count of unique smiles: 1423
Count of all of the smiles: 1423


In [8]:
from MLPipeline import MLmodel, BinTheTarget

Target = ['deltaG4_bin']
Features = ['SMILES']
Feature_types = ['SMILES']
input = df

In [12]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Name,SMILES,\DeltaG4,type,deltaG4_bin,deltaG4_realbin-27_-39
0,0,0,Carb_0_0_10,*[Ni@](C9=N([H])C(C(C)C)=C(C(C)C)N([H])9)(c1cc...,-19.000359,Carb,0,0
1,1,1,Carb_0_0_1,*[Ni@](C9=N([H])C(C)C(C)N([H])9)(c1ccc2ccccc2c...,-8.308853,Carb,1,0
2,2,2,Carb_0_0_25,*[Ni@](C9=N([H])C1C2=CC=CC(=C23)C=CC=C3C=1N([H...,-9.688746,Carb,1,0
3,3,3,Carb_0_0_7,*[Ni@](C9=N([H])C(C(F)(F)F)C(C(F)(F)F)N([H])9)...,-10.292410,Carb,1,0
4,4,4,Carb_0_11_0,*[Ni@](C9=N([H])CCN(C1CC1)9)(c1ccc2ccccc2c1)([...,-5.979538,Carb,1,0


In [9]:
def objectiveRandomForestClassifier(trial, model_instance):
    """
    Objective function for Optuna to minimize.
    """
    # Define hyperparameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_categorical('max_depth', [None, 10, 20, 30, 40]),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 6),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # Clone the model to ensure a fresh instance each trial
    model_clone = clone(model_instance.model)
    model_clone.set_params(**params)
    
    # Define the score metric
    scoring = 'accuracy'

    # Perform cross-validation
    scores = cross_val_score(model_clone, model_instance.X_train, model_instance.y_train, cv=model_instance.cv, scoring=scoring)

    # Return the average score across all folds
    return scores.mean()


def objectiveXGBClassifier(trial, model_instance):
    """
    Objective function for Optuna to minimize for XGBClassifier.
    """
    # Define hyperparameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),  # L1 regularization
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),  # L2 regularization
    }

    # Clone the model to ensure a fresh instance each trial
    model_clone = clone(model_instance.model)
    model_clone.set_params(**params)
    
    # Define the score metric
    scoring = 'accuracy'

    # Perform cross-validation
    scores = cross_val_score(model_clone, model_instance.X_train, model_instance.y_train, cv=model_instance.cv, scoring=scoring)

    # Return the average score across all folds
    return scores.mean()


In [10]:
MODEL_NAMES = ['RandomForestClassifier', 'XGBClassifier']
TARGETS = [[i] for i in Target ]
TRAIN_SIZES = [25, 50, 100, 200]

result = []
for seed in [1, 2, 3]:
    for model_name in MODEL_NAMES:
        for target in TARGETS:
            for train_size in TRAIN_SIZES:
                print(f'RUN: Model:{model_name} / Target:{target} / Train size:{train_size} / Seed:{seed}')
                
                if model_name == 'RandomForestClassifier':
                    objective = objectiveRandomForestClassifier
                elif model_name == 'XGBClassifier':
                    objective = objectiveXGBClassifier

                model = MLmodel(modelType=model_name, 
                df=input,
                randomSeed=seed,
                train_count = train_size,
                test_count = 50, 
                target=target, 
                features=Features, 
                hyperparameter_tuning=True,
                feature_types=Feature_types,
                optimization_method='optuna', 
                optimization_trials=10,
                objective=lambda trial: objective(trial, model)
                )

                model.train()
                eval, summary = model.evaluate()
                result.append({**summary, **eval, 'seed':seed})



RUN: Model:RandomForestClassifier / Target:['deltaG4_bin'] / Train size:25 / Seed:1


[11:03:42] Explicit valence for atom # 3 N, 4, is greater than permitted
[11:03:42] Explicit valence for atom # 3 N, 4, is greater than permitted
[11:03:42] Explicit valence for atom # 3 N, 4, is greater than permitted
[11:03:42] Explicit valence for atom # 3 N, 4, is greater than permitted
[11:03:42] Explicit valence for atom # 3 N, 4, is greater than permitted
[11:03:42] Explicit valence for atom # 3 N, 4, is greater than permitted
[11:03:42] Explicit valence for atom # 3 N, 4, is greater than permitted
[11:03:42] Explicit valence for atom # 3 N, 4, is greater than permitted
[11:03:42] Explicit valence for atom # 3 N, 4, is greater than permitted
[11:03:42] Explicit valence for atom # 3 N, 4, is greater than permitted
[11:03:42] Explicit valence for atom # 3 N, 4, is greater than permitted
[11:03:42] Explicit valence for atom # 3 N, 4, is greater than permitted
[11:03:42] Explicit valence for atom # 3 N, 4, is greater than permitted
[11:03:42] Explicit valence for atom # 3 N, 4, is g

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (25,) + inhomogeneous part.

In [ ]:
df = pd.DataFrame(result)
display(df)
df.to_csv('FreeEnergy_cleavageReact_traditional_ml.csv')

In [ ]:
import matplotlib.pyplot as plt

METRICS = ['accuracy','f1_micro','f1_macro', 'kappa']
fig, ax = plt.subplots(len(METRICS), 1, sharex=True, layout = 'constrained')
ax = ax.flatten()

for i, metric in enumerate(METRICS):
    ax[i].plot(df['train_size'], df[metric], marker = 'o')
    ax[i].set_title(metric)
    ax[i].set_ylim(0,1)

fig.suptitle('Random Forrest & XGB - FreeEnergy_cleavageReact')